# Buckling under lateral force: ABAQUS sims and plots

In [ ]:
import bikewheellib as bl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from doetools import LateralBucklingDOE, wheel_library
from IPython.display import display
from shutil import copy2
from imp import find_module
import matplotlib as mpl

%matplotlib inline

## ABAQUS Explicit dynamic

In [ ]:
exp_dir = '../data/doe/doe_lat_buckling_exp'

if False:

    doe_opts = {'spk_paired': False, 'spk_eltype': 'beam',
                'sim_type': 'exp', 'sim_u3': 0.04, 'sim_alpha': 1e3}

    doe = LateralBucklingDOE(out_dir=exp_dir)

    # Create wheel
    wheel = wheel_library.create_predefined_wheel('cheap_MTB')

    # Get buckling tension
    Tc_theor, nc = bl.calc_buckling_tension(wheel)

    for Tn in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]:
        jobname = 'Tn{:.2f}'.format(Tn)

        wheel = wheel_library.create_predefined_wheel('vintage_rd')
        
        doe_opts.update({'jobname': jobname, 'spk_Tn': Tn, 'spk_T': Tn * Tc_theor})
        doe.add_experiment(wheel, opts=doe_opts)

    print('\nCreated {0:d} simulations'.format(len(doe.db)))

    doe.write_input_files(N_batches=3)
    doe.to_csv()
    
    # Copy postprocessing script
    copy2(src=find_module('doetools')[1] + '/postproc_lat_buckling.py',
          dst=exp_dir)

## ABAQUS Standard Riks

In [ ]:
riks_dir = '../data/doe/doe_lat_buckling_riks'

if True:

    doe_opts = {'spk_paired': False, 'spk_eltype': 'beam',
                'sim_type': 'riks', 'sim_u3': 0.055, 'sim_Pmax': 500.}

    doe = LateralBucklingDOE(out_dir=riks_dir)

    # Create wheel
    wheel = wheel_library.create_predefined_wheel('vintage_rd')

    # Get buckling tension
    Tc_theor, nc = bl.calc_buckling_tension(wheel)

    for Tn in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]:
        jobname = 'Tn{:.2f}'.format(Tn)

        doe_opts.update({'jobname': jobname, 'spk_Tn': Tn,'spk_T': Tn*Tc_theor})
        doe.add_experiment(wheel, opts=doe_opts)

    print('\nCreated {0:d} simulations'.format(len(doe.db)))

    doe.write_input_files(N_batches=4)
    doe.to_csv()
    
    # Copy postprocessing script
    copy2(src=find_module('doetools')[1] + '/postproc_lat_riks.py',
          dst=riks_dir)

## Plots

In [ ]:
# Load DOE database
print 'Loading database...'
doe_r = LateralBucklingDOE(out_dir=riks_dir, db_file=riks_dir+'/_doe_db.csv')

# Populate DOE database with results
print 'Extracting results...'
doe_r.extract_results()
print '\nDone'

In [ ]:
def find_P_max(r):
    pd_data = pd.read_csv(riks_dir + '/' + r.name + '_riks_Pd.csv')
    
    P = pd_data['RF3 [N]']
    
    dP = P.values[1:] - P.values[:-1]
    return np.max(P[:np.argmax(dP < 0)+5])

doe_r.db['Pc_max'] = doe_r.db.apply(find_P_max, axis=1)

In [ ]:
cp = sns.color_palette('Reds', 7)

with plt.style.context('seaborn-paper'):

    fig, ax = plt.subplots(figsize=(2.5, 2.5))

    j = 0
    for i, r in doe_r.db[doe_r.db['spk_Tn'].between(0.1, 0.7)].iterrows():
        pd_data = pd.read_csv(riks_dir + '/' + r.name + '_riks_Pd.csv')

        u = 1000*(pd_data['U3 [m]'] - pd_data['U3 [m]'][0])
        P = pd_data['RF3 [N]']

        ax.plot(u, P, color=cp[j], label='{:.1f}'.format(r.spk_Tn))
        j += 1

        i_zero = np.where(np.diff(np.sign(P)))[0][1:]

        ax.plot(u[i_zero], P[i_zero], 'k*', label='_nolegend_', markersize=8)

    ax.set_xlim([0., 50])
    ax.set_xlabel('Lat. displacement [mm]')
    ax.set_ylabel('Lateral load [N]')

    ax.set_ylim([-50., 400.])
    ax.set_yticks(range(0, 401, 100))
    
    ax.set_xticks(range(0, 51, 10))

    plt.tight_layout()
    plt.savefig('../figs/buckling_ext_loads/_python_lat_riks.pdf')

In [ ]:
wheel = doe_r.wheel_from_row(doe_r.db.iloc[0])

Tc, nc = bl.calc_buckling_tension(wheel)

EA = wheel.spokes[0].EA
ls = wheel.spokes[0].length
a = wheel.spokes[0].alpha

TT = np.linspace(0., 0.999*Tc, 50)
Pc_the = [T / (EA) * ls/np.sin(a) * bl.lateral_stiffness(wheel, tension=T)
          for T in TT]

with plt.style.context('seaborn-paper'):
    fig, ax = plt.subplots(figsize=(2.5, 2.5))

    sns_grn = '#6acc65'  #sns.color_palette('muted', 5)[1]
    sns_red = '#ed6145'  #sns.color_palette('OrRd', 10)[6]
    sns_ylw = '#fedfb5'  #sns.color_palette('OrRd', 10)[1]

    ax.patch.set_facecolor('pink')

    ax.fill(np.concatenate(([0.], doe_r.db['spk_Tn'], [1.0])),
            np.concatenate(([0.], doe_r.db['Pc_max'], [0.])), color=sns_ylw)

    ax.fill(np.append(TT / max(TT), 1.0), np.append(Pc_the, 0.), color=sns_grn)

    ax.plot(np.append(TT / max(TT), 1.0), np.append(Pc_the, 0.), 'k--', label='spoke buckling (theory)')
    ax.plot(doe_r.db['spk_Tn'], doe_r.db['Pc_spk'], color='orange', marker='o', label='spoke buckling (FEA)')
    ax.plot(doe_r.db['spk_Tn'], doe_r.db['Pc_max'], color='red', marker='^', label='peak load (FEA)')

    ax.set_xlim([0, 0.95])
    ax.set_xticks([0., 0.25, 0.5, 0.75])
    ax.set_xticklabels(['0', '0.25', '0.5', '0.75'])
    ax.set_ylim([0., 400.])
    ax.set_yticks([0., 100, 200, 300, 400])

    ax.set_xlabel('\$T/T_c\$')
    ax.set_ylabel('Lateral load [N]')

    plt.tight_layout()
    plt.savefig('../figs/buckling_ext_loads/_python_lat_fail_map.pdf')